<a href="https://colab.research.google.com/github/rahulroynit/Bayesian-vs-Hyperband-Optimization/blob/master/BH_Regression_(Hyperband_vs_Bayesian).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# create set up for installing kerastuner
# installation needed only once
import os, sys
lib_path = '/content/libraries'
os.symlink('/content/drive/My Drive/Colab Notebooks', lib_path)
sys.path.insert(0, lib_path)

In [ ]:
# install keras-tuner
# needs to be run once for permanent installation in colab
!pip install --target=$lib_path keras-tuner

In [30]:
# import dependencies
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import Hyperband, BayesianOptimization
from kerastuner import HyperModel
from keras.datasets import boston_housing 
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import datetime
%load_ext tensorboard
pd.set_option('display.max_columns', None)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
# load dataset into train and evaluation data
# evaluation data will be split into test and valid data
(X_train, y_train), (X_eval, y_eval) = boston_housing.load_data()

In [32]:
# query train data
print(type(X_train), type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [33]:
# check dims of train and eval data
print('dims of train and eval data')
print(X_train.shape, X_eval.shape)

dims of train and eval data
(404, 13) (102, 13)


In [34]:
# check train data info
print(pd.DataFrame(X_train).info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       404 non-null    float64
 1   1       404 non-null    float64
 2   2       404 non-null    float64
 3   3       404 non-null    float64
 4   4       404 non-null    float64
 5   5       404 non-null    float64
 6   6       404 non-null    float64
 7   7       404 non-null    float64
 8   8       404 non-null    float64
 9   9       404 non-null    float64
 10  10      404 non-null    float64
 11  11      404 non-null    float64
 12  12      404 non-null    float64
dtypes: float64(13)
memory usage: 41.2 KB
None


In [35]:
# query train data samples
print(pd.DataFrame(X_train).head())
print(pd.DataFrame(y_train).head())

         0     1      2    3      4      5      6       7     8      9    10  \
0  1.23247   0.0   8.14  0.0  0.538  6.142   91.7  3.9769   4.0  307.0  21.0   
1  0.02177  82.5   2.03  0.0  0.415  7.610   15.7  6.2700   2.0  348.0  14.7   
2  4.89822   0.0  18.10  0.0  0.631  4.970  100.0  1.3325  24.0  666.0  20.2   
3  0.03961   0.0   5.19  0.0  0.515  6.037   34.5  5.9853   5.0  224.0  20.2   
4  3.69311   0.0  18.10  0.0  0.713  6.376   88.4  2.5671  24.0  666.0  20.2   

       11     12  
0  396.90  18.72  
1  395.38   3.11  
2  375.52   3.26  
3  396.90   8.01  
4  391.43  14.65  
      0
0  15.2
1  42.3
2  50.0
3  21.1
4  17.7


In [36]:
# split evaluation data into validation and test samples
from sklearn.model_selection import train_test_split
X_test, X_valid, y_test, y_valid = train_test_split(X_eval, y_eval, test_size = 0.5)

In [37]:
# normalize data using min-max scaling
# train, valid and test predictors are scaled using parameters of train predictors
# train and valid target values are scaled using parameters of train target 
# test target not scaled as it will be used for performance comparison on unseen data
# performance comparison to be done after unscaling predictions
y_train = y_train.reshape((-1, 1))
y_valid = y_valid.reshape((-1, 1))
scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
scaler_X.fit(X_train)
scaler_y.fit(y_train)
X_train_array = np.array(scaler_X.transform(X_train))
X_valid_array = np.array(scaler_X.transform(X_valid))
X_test_array = np.array(scaler_X.transform(X_test))
y_train_array = np.array(scaler_y.transform(y_train))
y_valid_array = np.array(scaler_y.transform(y_valid))

In [38]:
# check shape of validation data
print(X_valid_array.shape, y_valid_array.shape)

(51, 13) (51, 1)


In [39]:
# check test data shape
# test target should be rank-1 array, i.e., of the form: (N, )
print(type(X_test), type(y_test))
print(X_test.shape, y_test.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(51, 13) (51,)


In [40]:
# define model build
# define tunable hyperparameters using suitable hyperparameter types
# tunable hyperparameters in model: layers, neurons, dropout, learning rate
def build_model(hp):
  model = Sequential()
  model.add(Dense(units = hp.Int("dense_input", min_value = 32, max_value = 128, step = 32),
                                 input_shape = (X_train_array.shape[1],)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(Dropout(hp.Float("drop_input", min_value = 0, max_value = 0.3, step = 0.1)))
  for i in range(hp.Int("num_intermediate_layers", 1, 3)):
    model.add(Dense(units = hp.Int(f'dense_{i}', min_value = 32, max_value = 128, step = 32)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float(f'drop_{i}', min_value = 0, max_value = 0.3, step = 0.1)))
  model.add(Dense(1))
  model.compile(optimizer = Adam(learning_rate = hp.Float('lr', min_value = 0.0001, max_value = 0.01, sampling = "LOG", 
                                                          default = 0.01)), 
                loss = "mse", metrics = ["mae"])
  return(model)

In [41]:
## set up tensorboard
# clear logs from previous runs
!rm -rf ./logs/
# set up path where logs of tensorboard will be saved
# datetime suffix makes the logs unique
path = "logs/BH/" + datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
tensorboard = TensorBoard(log_dir = path, histogram_freq = 1) 

In [43]:
# select tuner for hyperparameter tuning (BayesianOptimization (BO) or Hyperband (HB))
tuner_choice = "HB"
# set max_trials (trials) for BO and max_epochs (ep) for HB
trials, ep = 10, 50
# set batch size
bs = 32

In [45]:
# set up tuner
if tuner_choice == "BO":
  tuner = BayesianOptimization(build_model, objective = "val_mae", max_trials = trials, executions_per_trial = 2, 
                               seed = 2020, project_name = "bo_tuning", overwrite = True)
else:
  tuner = Hyperband(build_model, objective = "val_mae", max_epochs = ep, executions_per_trial = 2,
                    seed = 2020, project_name = "hb_tuning", overwrite = True)

In [46]:
# tuner search space summary
print(tuner.search_space_summary())

None


In [47]:
# optimize hyperparameters
# we set up early stopping of tuning if validation loss doesn't improve after 5 epochs  
tuner.search(x = X_train_array, y = y_train_array, batch_size = bs, epochs = ep, verbose = 1, validation_data = (X_valid_array, y_valid_array), 
             callbacks = [EarlyStopping('val_loss', patience = 5), tensorboard])

Epoch 1/2
13/13 [==============================] - 0s 33ms/step - loss: 0.9232 - mae: 0.7446 - val_loss: 0.0975 - val_mae: 0.2536
Epoch 2/2
13/13 [==============================] - 0s 13ms/step - loss: 0.3806 - mae: 0.4861 - val_loss: 0.0557 - val_mae: 0.1808
Epoch 1/2
13/13 [==============================] - 0s 29ms/step - loss: 1.4711 - mae: 0.8696 - val_loss: 0.0799 - val_mae: 0.2292
Epoch 2/2
13/13 [==============================] - 0s 11ms/step - loss: 0.4698 - mae: 0.5268 - val_loss: 0.0720 - val_mae: 0.2104


Epoch 1/2
13/13 [==============================] - 0s 32ms/step - loss: 1.2823 - mae: 0.8687 - val_loss: 0.0607 - val_mae: 0.1787
Epoch 2/2
13/13 [==============================] - 0s 15ms/step - loss: 0.5609 - mae: 0.5636 - val_loss: 0.0478 - val_mae: 0.1660
Epoch 1/2
13/13 [==============================] - 0s 31ms/step - loss: 0.9060 - mae: 0.7088 - val_loss: 0.0512 - val_mae: 0.1802
Epoch 2/2
13/13 [==============================] - 0s 14ms/step - loss: 0.4380 - mae: 0.5071 - val_loss: 0.0324 - val_mae: 0.1302


Epoch 1/2
13/13 [==============================] - 0s 37ms/step - loss: 0.4471 - mae: 0.4556 - val_loss: 0.0735 - val_mae: 0.2149
Epoch 2/2
13/13 [==============================] - 0s 15ms/step - loss: 0.0557 - mae: 0.1798 - val_loss: 0.0353 - val_mae: 0.1385
Epoch 1/2
13/13 [==============================] - 0s 33ms/step - loss: 0.3445 - mae: 0.4109 - val_loss: 0.0413 - val_mae: 0.1516
Epoch 2/2
13/13 [==============================] - 0s 16ms/step - loss: 0.0563 - mae: 0.1797 - val_loss: 0.0321 - val_mae: 0.1304


Epoch 1/2
13/13 [==============================] - 0s 38ms/step - loss: 1.1634 - mae: 0.8002 - val_loss: 0.0724 - val_mae: 0.2195
Epoch 2/2
13/13 [==============================] - 0s 15ms/step - loss: 0.3907 - mae: 0.4912 - val_loss: 0.0514 - val_mae: 0.1708
Epoch 1/2
13/13 [==============================] - 0s 33ms/step - loss: 0.9977 - mae: 0.7374 - val_loss: 0.0798 - val_mae: 0.2239
Epoch 2/2
13/13 [==============================] - 0s 17ms/step - loss: 0.3623 - mae: 0.4567 - val_loss: 0.0418 - val_mae: 0.1527


Epoch 1/2
13/13 [==============================] - 0s 27ms/step - loss: 2.5115 - mae: 1.1398 - val_loss: 0.0519 - val_mae: 0.1658
Epoch 2/2
13/13 [==============================] - 0s 12ms/step - loss: 0.7597 - mae: 0.6652 - val_loss: 0.0288 - val_mae: 0.1293
Epoch 1/2
13/13 [==============================] - 0s 25ms/step - loss: 1.4950 - mae: 0.9013 - val_loss: 0.0816 - val_mae: 0.2337
Epoch 2/2
13/13 [==============================] - 0s 10ms/step - loss: 0.5450 - mae: 0.5747 - val_loss: 0.1107 - val_mae: 0.2765


...

Epoch 1/50
13/13 [==============================] - 0s 34ms/step - loss: 1.2383 - mae: 0.8561 - val_loss: 0.0739 - val_mae: 0.2157
Epoch 2/50
13/13 [==============================] - 0s 16ms/step - loss: 0.8257 - mae: 0.6910 - val_loss: 0.0510 - val_mae: 0.1700
Epoch 3/50
13/13 [==============================] - 0s 18ms/step - loss: 0.5546 - mae: 0.5600 - val_loss: 0.0376 - val_mae: 0.1400
Epoch 4/50
13/13 [==============================] - 0s 14ms/step - loss: 0.5200 - mae: 0.5591 - val_loss: 0.0424 - val_mae: 0.1490
Epoch 5/50
13/13 [==============================] - 0s 14ms/step - loss: 0.4076 - mae: 0.4899 - val_loss: 0.0392 - val_mae: 0.1407
Epoch 6/50
13/13 [==============================] - 0s 13ms/step - loss: 0.3899 - mae: 0.4787 - val_loss: 0.0440 - val_mae: 0.1543
Epoch 7/50
13/13 [==============================] - 0s 13ms/step - loss: 0.4350 - mae: 0.5045 - val_loss: 0.0392 - val_mae: 0.1425
Epoch 8/50
13/13 [==============================] - 0s 13ms/step - loss: 0.3927 - m

Epoch 1/50
13/13 [==============================] - 0s 27ms/step - loss: 2.7275 - mae: 1.2400 - val_loss: 0.4325 - val_mae: 0.6278
Epoch 2/50
13/13 [==============================] - 0s 12ms/step - loss: 1.8626 - mae: 1.0282 - val_loss: 0.2866 - val_mae: 0.4997
Epoch 3/50
13/13 [==============================] - 0s 12ms/step - loss: 1.7978 - mae: 1.0037 - val_loss: 0.1906 - val_mae: 0.3938
Epoch 4/50
13/13 [==============================] - 0s 12ms/step - loss: 1.4320 - mae: 0.8759 - val_loss: 0.1352 - val_mae: 0.3190
Epoch 5/50
13/13 [==============================] - 0s 12ms/step - loss: 1.2084 - mae: 0.8240 - val_loss: 0.1049 - val_mae: 0.2703
Epoch 6/50
13/13 [==============================] - 0s 12ms/step - loss: 0.9675 - mae: 0.7552 - val_loss: 0.0834 - val_mae: 0.2318
Epoch 7/50
13/13 [==============================] - 0s 12ms/step - loss: 0.9564 - mae: 0.7589 - val_loss: 0.0690 - val_mae: 0.2027
Epoch 8/50
13/13 [==============================] - 0s 12ms/step - loss: 1.0675 - m

Epoch 1/50
13/13 [==============================] - 0s 29ms/step - loss: 0.9518 - mae: 0.7205 - val_loss: 0.0425 - val_mae: 0.1558
Epoch 2/50
13/13 [==============================] - 0s 11ms/step - loss: 0.4129 - mae: 0.4694 - val_loss: 0.0481 - val_mae: 0.1676
Epoch 3/50
13/13 [==============================] - 0s 11ms/step - loss: 0.3045 - mae: 0.4006 - val_loss: 0.0538 - val_mae: 0.1800
Epoch 4/50
13/13 [==============================] - 0s 17ms/step - loss: 0.1986 - mae: 0.3400 - val_loss: 0.0390 - val_mae: 0.1462
Epoch 5/50
13/13 [==============================] - 0s 13ms/step - loss: 0.2127 - mae: 0.3362 - val_loss: 0.0426 - val_mae: 0.1556
Epoch 6/50
13/13 [==============================] - 0s 18ms/step - loss: 0.1999 - mae: 0.3332 - val_loss: 0.0357 - val_mae: 0.1387
Epoch 7/50
13/13 [==============================] - 0s 14ms/step - loss: 0.1391 - mae: 0.2806 - val_loss: 0.0359 - val_mae: 0.1379
Epoch 8/50
13/13 [==============================] - 0s 12ms/step - loss: 0.1298 - m

Epoch 1/50
13/13 [==============================] - 0s 37ms/step - loss: 1.4373 - mae: 0.9172 - val_loss: 0.0490 - val_mae: 0.1679
Epoch 2/50
13/13 [==============================] - 0s 15ms/step - loss: 0.6551 - mae: 0.6256 - val_loss: 0.0362 - val_mae: 0.1384
Epoch 3/50
13/13 [==============================] - 0s 13ms/step - loss: 0.6697 - mae: 0.6382 - val_loss: 0.0558 - val_mae: 0.1832
Epoch 4/50
13/13 [==============================] - 0s 14ms/step - loss: 0.4920 - mae: 0.5222 - val_loss: 0.0436 - val_mae: 0.1567
Epoch 5/50
13/13 [==============================] - 0s 13ms/step - loss: 0.3953 - mae: 0.4729 - val_loss: 0.0440 - val_mae: 0.1568
Epoch 6/50
13/13 [==============================] - 0s 14ms/step - loss: 0.4269 - mae: 0.4941 - val_loss: 0.0371 - val_mae: 0.1407
Epoch 7/50
13/13 [==============================] - 0s 12ms/step - loss: 0.4507 - mae: 0.4848 - val_loss: 0.0549 - val_mae: 0.1820
Epoch 1/50
13/13 [==============================] - 1s 46ms/step - loss: 1.4451 - m

INFO:tensorflow:Oracle triggered exit


In [48]:
# tuning summary
print(tuner.results_summary())

None


In [49]:
# best hyperparameter values 
best_hp = pd.DataFrame(tuner.get_best_hyperparameters()[0].values, index = [0])
print(best_hp)

   dense_input  drop_input  num_intermediate_layers  dense_0  drop_0  \
0           96         0.2                        2       32     0.1   

         lr  dense_1  drop_1  dense_2  drop_2  tuner/epochs  \
0  0.000319       96     0.0       96     0.1            50   

   tuner/initial_epoch  tuner/bracket  tuner/round  
0                    0              0            0  


In [50]:
# best model
best_model = tuner.get_best_models()[0]
print(best_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 96)                1344      
_________________________________________________________________
batch_normalization (BatchNo (None, 96)                384       
_________________________________________________________________
activation (Activation)      (None, 96)                0         
_________________________________________________________________
dropout (Dropout)            (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                3104      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0

In [51]:
# save best model
best_model.save(f'bh_model_{tuner_choice}.h5')

In [52]:
# load best model (if not loaded)
best_model = keras.models.load_model('bh_model_HB.h5')

In [53]:
# predictions on test data
# predictions are not rank-1 arrays
# predictions to be flattened after inverse transformation
preds = best_model.predict(x = X_test_array)
print(type(preds), preds.shape)
print(pd.DataFrame(preds).head())

<class 'numpy.ndarray'> (51, 1)
          0
0  0.698106
1  0.489736
2  0.369698
3  0.683863
4  0.290803


In [54]:
# unscale predictions and convert into rank-1 array
preds_unscaled = np.array(scaler_y.inverse_transform(preds)).flatten()
print(type(preds_unscaled), preds_unscaled.shape)

<class 'numpy.ndarray'> (51,)


In [55]:
# compare unscaled predictions and actual values
print(pd.DataFrame(preds_unscaled, columns = ['predictions']).head())
print(pd.DataFrame(y_test, columns = ['actual_values']).head())

   predictions
0    36.414768
1    27.038111
2    21.636400
3    35.773830
4    18.086123
   actual_values
0           43.1
1           26.6
2           20.5
3           36.0
4           19.8


In [56]:
# performance on test data
mape = np.mean(np.abs((np.array(y_test) - np.array(preds_unscaled))/np.array(y_test))) * 100
mae = np.mean(np.abs(np.array(y_test) - np.array(preds_unscaled)))
print(f'mape = {mape} \n', f'mae = {mae}')

mape = 14.957956681140017 
 mae = 2.833879139844109


In [ ]:
# run tensorboard
%tensorboard --logdir logs/BH